In [2]:
#Custom Environment
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib import style
import time
import numpy as np
import random


style.use("ggplot")


class Blob():
    def __init__(self, SIZE = 10):
        self.size = SIZE
        self.x = np.random.randint(0, SIZE)
        self.y = np.random.randint(0, SIZE)

    def __str__(self):
        return f"{self.x}, {self.y}"

    def __sub__(self, other):
        return (self.x-other.x, self.y-other.y)

    def act(self, choice, diagonal = False):
        '''
        Gives us 4 total movement options. (0,1,2,3)
        '''
        if diagonal:

            if choice == 0:
                self.move(x=1, y=1)
            elif choice == 1:
                self.move(x=-1, y=-1)
            elif choice == 2:
                self.move(x=-1, y=1)
            elif choice == 3:
                self.move(x=1, y=-1)

        else:
            if choice == 0:
                self.move(x=0, y=1)
            elif choice == 1:
                self.move(x=0, y=-1)
            elif choice == 2:
                self.move(x=-1, y=0)
            elif choice == 3:
                self.move(x=1, y=0)


    def move(self, x=-100, y=-100):

        if x == -100:
            self.x += np.random.randint(-1, 2)
        else:
            self.x += x

        if y == -100:
            self.y += np.random.randint(-1, 2)
        else:
            self.y += y

        if self.x < 0:
            self.x = 0
        elif self.x > self.size-1:
            self.x = self.size-1
        if self.y < 0:
            self.y = 0
        elif self.y > self.size-1:
            self.y = self.size-1


class ENVIRONMENT():



    def __init__(self, num_player=1, num_enemy=1, num_food=1, size = 10, diagonal = False):
        self.size = size
        self.naction = 4
        self.diagonal = diagonal
        self.num_enemy = num_enemy
        self.num_food = num_food
        self.player = Blob(size)
        self.enemy = [Blob() for _ in range(self.num_enemy)]
        self.food = [Blob() for _ in range(self.num_food)]
        self.reward = 0
        self.colors = {1: (255, 0, 0),
         2: (0, 255, 0),
         3: (0, 0, 255)}
        self.px,self.py = self.player.x,self.player.y
        self.ex,self.ey = [self.enemy[iter].x for iter in range(self.num_enemy)], [self.enemy[iter].y for iter in range(self.num_enemy)]
        self.fx,self.fy = [self.food[iter].x for iter in range(self.num_food)], [self.food[iter].y for iter in range(self.num_food)]


    def startover(self, newpos=False):

        self.player.x, self.player.y = self.px, self.py
        for iter in range(self.num_enemy):
            self.enemy[iter].x, self.enemy[iter].y = self.ex[iter], self.ey[iter]
        for iter in range(self.num_food):
            self.food[iter].x, self.food[iter].y = self.fx[iter], self.fy[iter]
        if newpos == True:
            self.player = Blob(self.size)
        self.reward = 0

        return (self.player.x, self.player.y), self.reward, False

    def step(self, action):

        self.player.act(action, self.diagonal)
        self.reward = self.calculate_reward()
        return (self.player.x, self.player.y), self.reward

    def calculate_reward(self):

        if self.player.x in [self.enemy[iter].x for iter in range(self.num_enemy)] and self.player.y in [self.enemy[iter].y for iter in range(self.num_enemy)]:
            return -100, True

        if self.player.x in [self.food[iter].x for iter in range(self.num_food)] and self.player.y in [self.food[iter].y for iter in range(self.num_food)]:
            return 100, True

        else:
            return -1, False


    def render(self,renderTime=100):

        env = np.zeros((self.size, self.size, 3), dtype=np.uint8)
        for iter in range(self.num_food):
            env[self.food[iter].x][self.food[iter].y] = self.colors[2]
        for iter in range(self.num_enemy):
            env[self.enemy[iter].x][self.enemy[iter].y] = self.colors[3]
        env[self.player.x][self.player.y] = self.colors[1]
        img = Image.fromarray(env, 'RGB')
        img = img.resize((300, 300))
        cv2.imshow("image", np.array(img))
        cv2.waitKey(renderTime)
        # cv2.destroyAllWindows()

    def sample_action(self):
        return np.random.randint(0, self.naction)

In [3]:
env = ENVIRONMENT(diagonal=True, size=10, num_enemy = 3, num_food = 1)
nS = 100
nA = 4
episodes = 25000
epsilon = 0.95
gamma = 0.9
learning_rate = 0.1
"""
Actions 
diagonal = True
0 = down_right
1 = up_left
2 = up_right
3 = down_left
When space is not available action = action.split('_')[0]

Environment
player = Blue
enemy = red
goal = green 

If a player is on 
an enemy reward at that time step = -100
the goal reward at that time step = 100
for every other time step reward is = -1
"""

"\nActions \ndiagonal = True\n0 = down_right\n1 = up_left\n2 = up_right\n3 = down_left\nWhen space is not available action = action.split('_')[0]\n\nEnvironment\nplayer = Blue\nenemy = red\ngoal = green \n\nIf a player is on \nan enemy reward at that time step = -100\nthe goal reward at that time step = 100\nfor every other time step reward is = -1\n"

In [5]:
Q = np.zeros([nS,nA])
for e in range(episodes):
    pos_i,r0,term = env.startover(newpos = True)
    s_i = pos_i[0]*10 + pos_i[1]
    a_i = E_policy(Q,s_i,epsilon)
    while term == False:
        #a_i = E_policy(Q,s_i,epsilon)
        pos_f,(r,term) = env.step(a_i)
        s_f = pos_f[0]*10 + pos_f[1]
        Q[s_i,a_i] = (1-learning_rate)*Q[s_i,a_i] + learning_rate*(r + gamma*np.max(Q[s_f]))
        a_i = E_policy(Q,s_f,epsilon)
        s_i = s_f
    if (e+1)%500 == 0:
        print("Epsiode = ",e)
    epsilon = epsilon*0.09998

Epsiode =  499
Epsiode =  999
Epsiode =  1499
Epsiode =  1999
Epsiode =  2499
Epsiode =  2999
Epsiode =  3499
Epsiode =  3999
Epsiode =  4499
Epsiode =  4999
Epsiode =  5499
Epsiode =  5999
Epsiode =  6499
Epsiode =  6999
Epsiode =  7499
Epsiode =  7999
Epsiode =  8499
Epsiode =  8999
Epsiode =  9499
Epsiode =  9999
Epsiode =  10499
Epsiode =  10999
Epsiode =  11499
Epsiode =  11999
Epsiode =  12499
Epsiode =  12999
Epsiode =  13499
Epsiode =  13999
Epsiode =  14499
Epsiode =  14999
Epsiode =  15499
Epsiode =  15999
Epsiode =  16499
Epsiode =  16999
Epsiode =  17499
Epsiode =  17999
Epsiode =  18499
Epsiode =  18999
Epsiode =  19499
Epsiode =  19999
Epsiode =  20499
Epsiode =  20999
Epsiode =  21499
Epsiode =  21999
Epsiode =  22499
Epsiode =  22999
Epsiode =  23499
Epsiode =  23999
Epsiode =  24499
Epsiode =  24999


In [4]:
def E_policy(q,s,epsilon):
    r = random.random()
    if r<epsilon:
        a = env.sample_action()
    else:
        a = np.argmax(q[s])
    return a

In [6]:
print(Q)
pol = np.zeros(nS)
for s in range(nS):
    pol[s] = np.argmax(Q[s])

[[-2.16037039e+00 -2.14321859e+00 -2.22585775e+00  1.51113719e+02]
 [ 1.50617440e+02 -2.27879625e+00 -2.27219241e+00 -2.21695641e+00]
 [-1.00000000e+01 -2.21379191e+00 -2.18052647e+00  1.89386045e+02]
 [-1.00000000e+01  1.68300021e+02 -2.26721949e+00 -2.28174245e+00]
 [-1.00000000e+01 -2.06674091e+00  1.89264742e+02 -1.00000000e+01]
 [ 2.12502780e+02 -1.77188608e+00 -1.73478528e+00 -1.00000000e+01]
 [ 1.89046170e+02 -1.64097722e+00 -1.62352429e+00 -1.00000000e+01]
 [-1.44588738e+00 -1.51097746e+00 -1.45178007e+00  2.11869457e+02]
 [-1.45648349e+00 -1.51788125e+00 -1.48552090e+00  1.89087864e+02]
 [-1.38180324e+00 -1.39937997e+00 -1.39941645e+00  2.11441968e+02]
 [-2.00526168e+00 -2.01126039e+00 -2.02121750e+00  1.69993816e+02]
 [ 2.12229691e+02 -1.97390149e+00 -1.98791846e+00 -1.91569721e+00]
 [-1.00000000e+01 -2.12769346e+00 -2.07549470e+00  1.69834547e+02]
 [-1.00000000e+01 -2.58351635e-01 -2.76506710e-01  2.11860070e+02]
 [-9.97680476e+00 -5.40849443e-01  1.89014333e+02 -9.91243474e

In [7]:
def play():
    pos_i,k,ter = env.startover(newpos=True)
    env.render()
    T = False
    i = 0
    while T == False and i<=20:   
        s = pos_i[0]*10 + pos_i[1]
        print(pos_i)
        pos_i,R = env.step(pol[s])
        print(R)
        T=R[1]
        env.render(500)
        i = i+1
    cv2.destroyAllWindows()

In [8]:
for i in range(10):
    play()

(1, 0)
(-1, False)
(2, 0)
(-1, False)
(3, 1)
(-1, False)
(4, 0)
(-1, False)
(5, 1)
(-1, False)
(6, 2)
(-1, False)
(7, 3)
(-1, False)
(8, 4)
(-1, False)
(9, 5)
(-1, False)
(8, 6)
(-1, False)
(9, 7)
(100, True)
(6, 8)
(-1, False)
(5, 9)
(-1, False)
(6, 9)
(-1, False)
(7, 8)
(-1, False)
(8, 7)
(100, True)
(8, 0)
(-1, False)
(7, 1)
(-1, False)
(6, 2)
(-1, False)
(7, 3)
(-1, False)
(8, 4)
(-1, False)
(9, 5)
(-1, False)
(8, 6)
(-1, False)
(9, 7)
(100, True)
(5, 0)
(-1, False)
(4, 0)
(-1, False)
(5, 1)
(-1, False)
(6, 2)
(-1, False)
(7, 3)
(-1, False)
(8, 4)
(-1, False)
(9, 5)
(-1, False)
(8, 6)
(-1, False)
(9, 7)
(100, True)
(6, 1)
(-1, False)
(7, 0)
(-1, False)
(6, 0)
(-1, False)
(5, 1)
(-1, False)
(6, 2)
(-1, False)
(7, 3)
(-1, False)
(8, 4)
(-1, False)
(9, 5)
(-1, False)
(8, 6)
(-1, False)
(9, 7)
(100, True)
(8, 5)
(-1, False)
(9, 6)
(-1, False)
(8, 7)
(100, True)
(1, 4)
(-1, False)
(0, 5)
(-1, False)
(1, 6)
(-1, False)
(2, 7)
(-1, False)
(3, 8)
(-1, False)
(4, 7)
(-1, False)
(5, 6)
(-1, 

In [10]:
pol.shape

(100,)